# Introduction


Utiliser l'environnement _emigration_, cf. les instructions de la page d'accueil du dépôt et le fichier _environment.yml_

In [ ]:
import mammoth

import pprint, re, os
from lxml import etree as etree

## Création de fonctions locales

In [ ]:
### Fonction qui marche, mettre toujours i = 0 au début – récursive

# Attention: tous les p avant le premier sous-h ne sont pas pris en compte
# dans la fonction, actuellement.
# De même à la fin d'une division il n'y a pas, par définition, de p hors h

def traitement_h(parent_childrens_list, parent_el, i):
    i = i + 1
    h = 'h' + str(i)
    if h in [ e[1] for e in parent_childrens_list]:
        for e in parent_childrens_list:
            if e[1] == h:
                id_e = e[0]
                div = etree.Element("div")
                div.append(e[2])

                ## produce element childrens list
                childs_list = []
                for child in parent_childrens_list:
                    if child[0] > id_e:
                        if child[1] == h:
                            break
                        else: 
                            childs_list.append(child)

                ### recursive treatment of following hs: h2, h3, ...
                traitement_h(childs_list, div, i)
                
                parent_el.append(div)
                
    else:
        [parent_el.append(e[2]) for e in parent_childrens_list]
    
    return parent_el

# Importation du document word et production des *spans* HTML

In [ ]:
### Doc mammoth :
# https://github.com/mwilliamson/python-mammoth
# https://recordnotfound.com/mammoth-js-mwilliamson-35447
# https://www.npmjs.com/package/mammoth_custom


# explications utiles au sujets des runs dans Al Sweigart, Automate, 'run attributes'
# https://automatetheboringstuff.com/chapter13/ (n'utilise pas mammoth mais docx)
# r = runs: styles dans le texte, au dessous du paragraphe ; p = styles de paragraphes

## Définit le mapping entre les styles Word et les éléments XML (tags) de sortie
style_map ="""
p[style-name='Heading 2'] => h2.test:fresh
r[style-name='semAnn_Segment'] => span.semAnn
"""

# p[style-name='Heading 2'] => h2:fresh

#### Définir les chemins où se trouvent les fichiers

In [ ]:
### Chemin des documents en .docx
folder_path = 'fichiers/'
### Chemin du dossier des documents transformes 
folder_path_xml = 'fichiers_xml/'


In [ ]:
l_docs = []
for d in os.listdir(folder_path):
    # vérifie que le document ne commence pas avec un '.', particularité MacOS pour docs ouverts
    if d[0] != '.':
        if('.doc' in d):
            l_docs.append(d)



In [ ]:
l_docs

In [ ]:
u_e = [
    {'from': '{', 'to':'<span class="tags">'},
    {'from': '}', 'to':'</span>'},
    {'from': '(\|)([0-9\-]{0,10})(\|)', 'to':'<date when="\g<2>" />'},
    {'from': '(§)(\s{0,2})(.{0,20})(\/)(.{0,20})(\s{0,2})(§)', 'to':'<rs class="hoover" ref="\g<5>">\g<3></rs>'}
    
]

In [ ]:
for doc in l_docs:
    doc_name = doc.replace('.docx', '')
    doc_address = folder_path + doc_name + '.docx'
    xml_doc_address = folder_path_xml + doc_name + '.html'

    with open(doc_address, "rb") as docx_file:
        print('\n————\n', xml_doc_address, '\n')
        try:
            result = mammoth.convert_to_html(docx_file, style_map=style_map)
            html = result.value # The generated HTML
            messages = result.messages # Any messages, such as warnings during conversion
        except Exception as e:
            print(e)
    print( messages) 
    
    html_doc = '<body>' + html + '</body>'
    lxml_root = etree.XML(html_doc)
    es = lxml_root.xpath('/body/*')

    es_list = []
    i = 1
    for e in es:
        es_list.append([i, e.tag, e])
        i = i + 1
    root = etree.Element("body")
    a = traitement_h(es_list, root, 0)
    a_string = etree.tostring(a,pretty_print=True,encoding='unicode')

    for l in u_e:
        p = re.compile(l['from'])
        txt_1 = (p.sub(l['to'], a_string))
        a_string = txt_1
    
    # Enlevé: '; charset=utf-8'
    html_doc_complet = """<html><head><title>""" + doc_name + """</title>
    <meta http-equiv="Content-Type" content="text/html"/>
<link rel="stylesheet" href="style.css" type="text/css"/></head>""" + a_string + '</html>'  
    # print(html_doc_complet[:1200])
    
    with open(xml_doc_address, 'w') as fichier:  
        fichier.write(html_doc_complet)

## Exploration des documents produits

Cf. le carnet 'textes_encodes_analyse'